# Tensorflow Road Signs YOLO Demo

>⚠️ **Warning:** This demo assumes that you have access to an on-prem deployment of Dioptra that provides a copy of the Road Signs dataset and a CUDA-compatible GPU.
> This demo cannot be run on a typical personal computer.

The demo provided in the Jupyter notebook `demo.ipynb` contains an example of how to set up and train a model based on the YOLO v1 architecture and use it to perform object detection on the Road Signs dataset.

## Setup

In [96]:
# Import packages from the Python standard library
import os
import pprint
import random
import time
import warnings
from pathlib import Path
from typing import Tuple

# Please enter custom username here.
USERNAME = "jglasbrenner"

# Filter out warning messages
warnings.filterwarnings("ignore")

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Testbed API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    # f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    f"http://restapi:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the AI_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["AI_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    # f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    f"http://mlflow-tracking:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to custom task plugins archives
CUSTOM_PLUGINS_EVALUATION_TAR_GZ = Path("custom-plugins-evaluation.tar.gz")
CUSTOM_PLUGINS_ROADSIGNS_YOLO_ESTIMATORS_TAR_GZ = Path("custom-plugins-roadsigns_yolo_estimators.tar.gz")

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_roadsigns_yolo"

# Path to dataset
data_path_roadsigns = "../../../data/roadsigns/images"
weights_path_yolo = "/home/jovyan/weights/yolo"

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Dataset

The Road Signs dataset needed for this demo can be obtained here: https://makeml.app/datasets/road-signs.
Object bounding boxes are provided in the PascalVOC format, which organizes the dataset into two folders:

    annotations/  (xml files)
    images/       (png files)

The PascalVOC format uses filenames to associate images with their corresponding annotation.
For example, the image file `images/road1.png` has an associated annotation file `annotations/road1.xml`.

This dataset does not provide a testing dataset, so we need to create our own.
We do this by stratifying over the number of objects and classes in the images and sampling 10% of the images without replacement within each of these groups.
The exact train/test split we used is reported in the `roadsigns_train_test_split.csv` file.

After performing our train/test split, our data is reorganized into the following folder structure:

    roadsigns
    ├── testing
    │   ├── annotations
    │   └── images
    └── training
        ├── annotations
        └── images

Let's take a look at a sampling of the training images to confirm that there aren't any problems,

In [98]:
roadsigns_training_samples = sorted(
    rng.choice(list(Path(data_path_roadsigns).glob("*.png")), size=4).tolist()
)
utils.notebook_gallery(roadsigns_training_samples, row_height="140px")

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within the testbed architecture, we need to package those files up into an archive and submit it to the Testbed RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make workflows

To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Testbed RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `AI_RESTAPI_URI` to figure out how to connect to the Testbed RESTful API,

In [ ]:
restapi_client = utils.SecuringAIClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Testbed API and returns a list of active queues.

In [ ]:
restapi_client.list_queues()

In [ ]:
response_queue = restapi_client.get_queue_by_name(name="tensorflow_cpu")

if response_queue is None or "Not Found" in response_queue.get("message", []):
    response_queue = restapi_client.register_queue(name="tensorflow_cpu")

response_queue

This example also makes use of the `evaluation` and `roadsigns_yolo_estimators` packages stored locally under the `task-plugins/securingai_custom` directory.
To register these custom task plugins, we first need to package them up into an archive.
For convenience, the `Makefile` provides a rule for creating the custom task plugins archive file, just run `make custom-plugins`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make custom-plugins

Now that the custom task plugin packages are packaged into archive files, next we register them by uploading the files to the REST API.
Note that we need to provide the name to use for custom task plugin package, and this name must be unique under the custom task plugins namespace.
For a full list of the custom task plugins, use `restapi_client.restapi_client.list_custom_task_plugins()`.

In [ ]:
response_evaluation_custom_plugins = restapi_client.get_custom_task_plugin(name="evaluation")

if response_evaluation_custom_plugins is None or "Not Found" in response_evaluation_custom_plugins.get("message", []):
    response_evaluation_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="evaluation",
        custom_plugin_file=CUSTOM_PLUGINS_EVALUATION_TAR_GZ,
    )

response_evaluation_custom_plugins

In [ ]:
response_roadsigns_custom_plugins = restapi_client.get_custom_task_plugin(name="roadsigns_yolo_estimators")

if response_roadsigns_custom_plugins is None or "Not Found" in response_roadsigns_custom_plugins.get("message", []):
    response_roadsigns_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_roadsigns_plugin_name="roadsigns_yolo_estimators",
        custom_roadsigns_plugin_file=CUSTOM_PLUGINS_ROADSIGNS_YOLO_ESTIMATORS_TAR_GZ,
    )

response_roadsigns_custom_plugins

If at any point you need to update one or more files within the `evaluation` or `roadsigns_yolo_estimators` packages, you will need to unregister/delete the custom task plugin first using the REST API.
This can be done as follows,

```python
# Delete the 'evaluation' custom task plugin package
restapi_client.delete_custom_task_plugin(name="evaluation")

# Delete the `roadsigns_yolo_estimators` package
restapi_client.delete_custom_task_plugin(name="roadsigns_yolo_estimators")
```

After you have deleted the task plugin in the testbed, re-run the `make custom-plugins` code block to update the package archive, then upload the updated plugin by re-running the `restapi_client.upload_custom_plugin_packge` block.

Next, we need to use transfer learning to update the object detection layers in our model.

```python
# Submit transfer learning job for the mobilenet_v2 + yolo network architecture
response_mobilenet_v2_yolo_transfer_learn = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="transfer_learn",
    entry_point_kwargs=" ".join([
        "-P data_dir=/nfs/data/roadsigns/training",
        "-P model_architecture=mobilenet_v2",
        "-P epochs=300",
        "-P batch_size=32",
        f"-P register_model_name={EXPERIMENT_NAME}_mobilenet_v2_yolo",
    ]),
)

print("Transfer learning job for MobileNet V2 + YOLO neural network submitted")
print("")
pprint.pprint(response_mobilenet_v2_yolo_transfer_learn)
```